In [ ]:
from pathlib import Path

import pandas as pd
import xarray as xr
from dask.distributed import Client
from seapopym.configuration.no_transport import ForcingParameter, ForcingUnit
from seapopym.standard.units import StandardUnitsLabels

from seapopym_optimization.cost_function import DayCycle, SimpleRootMeanSquareErrorCostFunction, TimeSeriesObservation
from seapopym_optimization.functional_group import NoTransportFunctionalGroup, Parameter
from seapopym_optimization.genetic_algorithm import SimpleGeneticAlgorithm, SimpleGeneticAlgorithmParameters
from seapopym_optimization.genetic_algorithm.simple_logbook import generate_logbook_with_sobol_sampling
from seapopym_optimization.model_generator import NoTransportModelGenerator


In [2]:
path_to_forcing = "../../../1_data_processing/1_1_Forcing/data/1_products/Hot_cmems_climato.zarr"
path_to_npp = "../../../1_data_processing/1_1_Forcing/data/1_products/Hot_observed_npp_climato.zarr"
path_to_obs = "../../../1_data_processing/1_1_Forcing/data/1_products/Hot_obs_zoo_climato_monthly_2002_2015.zarr"
export_file_name = "SeapoPym_HOT_climato_obs_npp_opti_all_parameters_1_group_night"

In [3]:
LATITUDE = 22.75
LONGITUDE = -158
TIME_START = "2005-01-02"
TIME_END = "2009-12-27"
STABILIZATION_TIME = 5
SAVE = True

## Loading


### Forcing


In [4]:
forcing = xr.open_zarr(path_to_forcing)
forcing = forcing.sel(time=slice(TIME_START, TIME_END))
forcing["T"].attrs["units"] = StandardUnitsLabels.temperature.units
forcing.load()

<xarray.Dataset> Size: 321kB
Dimensions:              (time: 1821, latitude: 1, longitude: 1, depth: 3)
Coordinates:
  * depth                (depth) int64 24B 0 1 2
  * latitude             (latitude) float64 8B 22.75
  * longitude            (longitude) int64 8B -158
  * time                 (time) datetime64[ns] 15kB 2005-01-02 ... 2009-12-27
Data variables: (12/13)
    T                    (time, latitude, longitude, depth) float64 44kB 23.8...
    U                    (time, latitude, longitude, depth) float64 44kB -0.0...
    V                    (time, latitude, longitude, depth) float64 44kB -0.0...
    mnkc_epi             (time, latitude, longitude) float64 15kB 0.2862 ... ...
    mnkc_hmlmeso         (time, latitude, longitude) float64 15kB 0.8062 ... ...
    mnkc_lmeso           (time, latitude, longitude) float64 15kB 2.113 ... 2...
    ...                   ...
    mnkc_mumeso          (time, latitude, longitude) float64 15kB 0.5676 ... ...
    mnkc_umeso           (time, latitude, longitude) float64 15kB 0.4922 ... ...
    npp                  (time, latitude, longitude) float64 15kB 322.7 ... 3...
    pelagic_layer_depth  (time, latitude, longitude, depth) float64 44kB 113....
    zeu                  (time, latitude, longitude) float64 15kB 75.51 ... 7...
    zooc                 (time, latitude, longitude) float64 15kB 0.267 ... 0...
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-10-17
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Global ocean low and mid trophic levels biomass content hin...

### Epipelagic layer


In [5]:
epi_layer_depth = forcing["pelagic_layer_depth"].sel(depth=0).load()
epi_layer_depth = epi_layer_depth.resample(time="1D").mean()
epi_layer_depth.attrs["units"] = "meter"
epi_layer_depth = epi_layer_depth.pint.quantify()
epi_layer_depth

Magnitude,[[[113.25850445781492]] [[113.67641789059907]] [[113.3777614275639]] ... [[112.86924868047693]] [[113.44111669274305]] [[111.81958173122077]]]
Units,meter


<!-- ## Observed NPP -->


In [6]:
observed_npp = xr.open_zarr(path_to_npp)
observed_npp = observed_npp.sel(time=slice(TIME_START, TIME_END))
observed_npp = observed_npp.dropna("time", how="all")
observed_npp = observed_npp.resample(time="D").interpolate("linear")
observed_npp.load()

<xarray.Dataset> Size: 42kB
Dimensions:          (time: 1765, latitude: 1, longitude: 1)
Coordinates:
  * latitude         (latitude) float64 8B 22.75
  * longitude        (longitude) int64 8B -158
  * time             (time) datetime64[ns] 14kB 2005-01-31 ... 2009-11-30
Data variables:
    l12              (time, latitude, longitude) float64 14kB 273.4 ... 275.3
    l12_lowess_0.05  (time, latitude, longitude) float64 14kB 274.7 ... 277.4
Attributes:
    desc:     L12 multiplied by ZEU (from CMEMS).
    type:     climatology

### Observations


In [7]:
observations = xr.open_zarr(path_to_obs).load()
observations = observations.sel(latitude=LATITUDE, longitude=LONGITUDE, method="nearest")
observations = observations.resample(time="1D").mean().dropna("time")
observations = observations.pint.quantify().pint.to("mg/m^3")
observations = observations * epi_layer_depth
observations = observations.drop_vars("depth")
observations

Magnitude,[[[[157.92219924943342]]] [[[179.50290001191829]]] [[[206.9568833881889]]] [[[265.77390000839995]]] [[[281.78101853068074]]] [[[282.22919058899055]]] [[[295.88177275463875]]] [[[313.19485204421545]]] [[[314.33474301409973]]] [[[252.30876534149797]]] [[[204.84900876732033]]] [[[189.26359852904628]]] [[[157.92219924943342]]] [[[179.50290001191829]]] [[[206.9568833881889]]] [[[265.77390000839995]]] [[[281.78101853068074]]] [[[282.22919058899055]]] [[[295.88177275463875]]] [[[313.19485204421545]]] [[[314.33474301409973]]] [[[252.30876534149797]]] [[[204.84900876732033]]] [[[189.26359852904628]]] [[[157.92219924943342]]] [[[179.50290001191829]]] [[[206.9568833881889]]] [[[265.77390000839995]]] [[[281.78101853068074]]] [[[282.22919058899055]]] [[[295.88177275463875]]] [[[313.19485204421545]]] [[[314.33474301409973]]] [[[252.30876534149797]]] [[[204.84900876732033]]] [[[189.26359852904628]]] [[[157.92219924943342]]] [[[178.58649139474625]]] [[[207.78348092804256]]] [[[265.6684200378455]]] [[[283.3215285687847]]] [[[286.8754053335449]]] [[[298.4606291423159]]] [[[316.99822292914615]]] [[[307.4117305615572]]] [[[256.3939882358341]]] [[[206.1695339111292]]] [[[184.04306987771415]]] [[[157.92219924943342]]] [[[179.50290001191829]]] [[[206.9568833881889]]] [[[265.77390000839995]]] [[[281.78101853068074]]] [[[282.22919058899055]]] [[[295.88177275463875]]] [[[313.19485204421545]]] [[[314.33474301409973]]] [[[252.30876534149797]]] [[[204.84900876732033]]]]
Units,milligram/meter2
Magnitude,[[[[237.0516536022709]]] [[[301.6376319958857]]] [[[337.83226845879244]]] [[[414.48959622350367]]] [[[389.1909678368789]]] [[[421.89365674709336]]] [[[424.06609922351373]]] [[[426.77207432265106]]] [[[418.1004648101298]]] [[[374.8468152741032]]] [[[333.9713931237319]]] [[[290.2018113694168]]] [[[237.0516536022709]]] [[[301.6376319958857]]] [[[337.83226845879244]]] [[[414.48959622350367]]] [[[389.1909678368789]]] [[[421.89365674709336]]] [[[424.06609922351373]]] [[[426.77207432265106]]] [[[418.1004648101298]]] [[[374.8468152741032]]] [[[333.9713931237319]]] [[[290.2018113694168]]] [[[237.0516536022709]]] [[[301.6376319958857]]] [[[337.83226845879244]]] [[[414.48959622350367]]] [[[389.1909678368789]]] [[[421.89365674709336]]] [[[424.06609922351373]]] [[[426.77207432265106]]] [[[418.1004648101298]]] [[[374.8468152741032]]] [[[333.9713931237319]]] [[[290.2018113694168]]] [[[237.0516536022709]]] [[[300.09769406058746]]] [[[339.1815897155657]]] [[[414.32509417720263]]] [[[391.31869310318103]]] [[[428.83910602723876]]] [[[427.7621889102778]]] [[[431.9547025535618]]] [[[408.89208174518643]]] [[[380.916096258274]]] [[[336.1242842926303]]] [[[282.197066227207]]] [[[237.0516536022709]]] [[[301.6376319958857]]] [[[337.83226845879244]]] [[[414.48959622350367]]] [[[389.1909678368789]]] [[[421.89365674709336]]] [[[424.06609922351373]]] [[[426.77207432265106]]] [[[418.1004648101298]]] [[[374.8468152741032]]] [[[333.9713931237319]]]]
Units,milligram/meter2


observations


Select the kind of observation you want to use.


In [8]:
# observations_selected = observations[["day_lowess_0.2", "night_lowess_0.2"]].rename(
#     {"day_lowess_0.2": "day", "night_lowess_0.2": "night"}
# )

observations_selected = observations["night"]

Remove the X first months to let the model reach the stationary state.


In [9]:
observations_selected_without_init = observations_selected.isel(time=slice(STABILIZATION_TIME, None))
observations_selected_without_init

Magnitude,[[[[421.89365674709336]]] [[[424.06609922351373]]] [[[426.77207432265106]]] [[[418.1004648101298]]] [[[374.8468152741032]]] [[[333.9713931237319]]] [[[290.2018113694168]]] [[[237.0516536022709]]] [[[301.6376319958857]]] [[[337.83226845879244]]] [[[414.48959622350367]]] [[[389.1909678368789]]] [[[421.89365674709336]]] [[[424.06609922351373]]] [[[426.77207432265106]]] [[[418.1004648101298]]] [[[374.8468152741032]]] [[[333.9713931237319]]] [[[290.2018113694168]]] [[[237.0516536022709]]] [[[301.6376319958857]]] [[[337.83226845879244]]] [[[414.48959622350367]]] [[[389.1909678368789]]] [[[421.89365674709336]]] [[[424.06609922351373]]] [[[426.77207432265106]]] [[[418.1004648101298]]] [[[374.8468152741032]]] [[[333.9713931237319]]] [[[290.2018113694168]]] [[[237.0516536022709]]] [[[300.09769406058746]]] [[[339.1815897155657]]] [[[414.32509417720263]]] [[[391.31869310318103]]] [[[428.83910602723876]]] [[[427.7621889102778]]] [[[431.9547025535618]]] [[[408.89208174518643]]] [[[380.916096258274]]] [[[336.1242842926303]]] [[[282.197066227207]]] [[[237.0516536022709]]] [[[301.6376319958857]]] [[[337.83226845879244]]] [[[414.48959622350367]]] [[[389.1909678368789]]] [[[421.89365674709336]]] [[[424.06609922351373]]] [[[426.77207432265106]]] [[[418.1004648101298]]] [[[374.8468152741032]]] [[[333.9713931237319]]]]
Units,milligram/meter2


Create structure for SeapoPym simulation.


In [10]:
forcing_parameters = ForcingParameter(
    temperature=ForcingUnit(forcing=forcing["T"]), primary_production=ForcingUnit(forcing=observed_npp["l12"])
)

2025-06-11 11:24:24,440 :: Seapodym ::  WARNING ::
|	l12 unit is milligram / day / meter ** 2, it will be converted to kilogram / day / meter ** 2.

2025-06-11 11:24:24,442 :: Seapodym ::  WARNING ::
|	l12 unit is milligram / day / meter ** 2, it will be converted to kilogram / day / meter ** 2.



## Setup the parameters and the cost function


In [11]:
functional_groups = [
    NoTransportFunctionalGroup(
        name="Zooplankton",
        day_layer=0,
        night_layer=0,
        energy_transfert=Parameter("D1N1_energy_transfert energy_transfert", 0.001, 0.5),
        gamma_tr=Parameter("D1N1_gamma_tr", -0.05, -0.001),
        tr_0=Parameter("D1N1_tr_0", 0, 50),
        gamma_lambda_temperature=Parameter("D1N1_gamma_lambda_temperature", 0, 0.5),
        lambda_temperature_0=Parameter("D1N1_lambda_temperature_0", 0, 1),
    ),
]

In [12]:
model_generator = NoTransportModelGenerator(forcing_parameters=forcing_parameters)

In [13]:
cost_function = SimpleRootMeanSquareErrorCostFunction(
    model_generator=model_generator,
    observations=[
        TimeSeriesObservation(
            name="Hot climato",
            observation=observations_selected_without_init,
            observation_type=DayCycle.NIGHT,
            observation_interval="1MS",
        )
    ],
    functional_groups=functional_groups,
    root_mse=True,
    normalized_mse=True,
    centered_mse=False,
)

Set the genetic algorithm meta parameters.


In [14]:
genetic_algo_parameters = SimpleGeneticAlgorithmParameters(
    MUTPB=0.30,
    INDPB=0.2,
    ETA=5,
    CXPB=0.7,
    NGEN=5,
    POP_SIZE=200,
    cost_function_weight=(-1,),
)

Import or create the parameters to be optimized.


In [19]:
logbook_path = Path(f"./{export_file_name}_logbook.parquet")
if logbook_path.exists():
    logbook = pd.read_parquet(logbook_path)
else:
    logbook = generate_logbook_with_sobol_sampling(
        functional_group_parameters=functional_groups, sample_number=256, fitness_name=["Hot climato"]
    )
logbook

category                                                                       Parametre  ... Weighted_fitness
name                                              D1N1_energy_transfert energy_transfert  ... Weighted_fitness
Generation Is_From_Previous_Generation Individual                                         ...                 
0          False                       0                                        0.149333  ...              NaN
                                       1                                        0.143150  ...              NaN
                                       2                                        0.149333  ...              NaN
                                       3                                        0.149333  ...              NaN
                                       4                                        0.149333  ...              NaN
...                                                                                  ...  ...              ...
                                       1787                                     0.151812  ...              NaN
                                       1788                                     0.151812  ...              NaN
                                       1789                                     0.151812  ...              NaN
                                       1790                                     0.151812  ...              NaN
                                       1791                                     0.265975  ...              NaN

[1792 rows x 7 columns]

Finaly, create the Genetic Algorithm.


In [20]:
client = Client()
genetic_algo = SimpleGeneticAlgorithm(
    meta_parameter=genetic_algo_parameters,
    cost_function=cost_function,
    client=client,
    logbook=logbook,
    save=logbook_path,
)

And watch the magic on the Dask dashboard :


In [21]:
genetic_algo.client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 48.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:62541,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:62553,Total threads: 3
Dashboard: http://127.0.0.1:62557/status,Memory: 12.00 GiB
Nanny: tcp://127.0.0.1:62544,


## Run the optimization


In [22]:
viewer = genetic_algo.optimize()

Some individuals in the logbook have no fitness values. Re-evaluating the population.


## Optimization statistics


In [26]:
viewer.hall_of_fame().iloc[:50]

category                                                                       Parametre  ... Weighted_fitness
name                                              D1N1_energy_transfert energy_transfert  ... Weighted_fitness
Generation Is_From_Previous_Generation Individual                                         ...                 
4          False                       149                                      0.464474  ...        -0.356171
0          False                       1642                                     0.389118  ...        -0.375671
3          False                       26                                       0.416938  ...        -0.401781
0          False                       1643                                     0.389118  ...        -0.402009
                                       76                                       0.486999  ...        -0.410396
4          False                       152                                      0.416938  ...        -0.412185
3          False                       101                                      0.433758  ...        -0.412382
4          False                       114                                      0.263351  ...        -0.420332
                                       190                                      0.433758  ...        -0.441438
                                       86                                       0.437408  ...        -0.448674
                                       66                                       0.437408  ...        -0.452810
                                       119                                      0.433758  ...        -0.462675
2          False                       110                                      0.381553  ...        -0.469915
0          False                       1389                                     0.357045  ...        -0.478403
3          False                       85                                       0.437408  ...        -0.481060
4          False                       85                                       0.499872  ...        -0.484349
                                       55                                       0.461408  ...        -0.485613
0          False                       158                                      0.315724  ...        -0.497578
                                       1184                                     0.484801  ...        -0.498722
1          False                       130                                      0.437408  ...        -0.511920
3          False                       93                                       0.349232  ...        -0.513585
0          False                       154                                      0.315724  ...        -0.515831
                                       462                                      0.485486  ...        -0.521548
                                       467                                      0.485486  ...        -0.521548
4          False                       71                                       0.446372  ...        -0.525663
0          False                       716                                      0.363787  ...        -0.526668
                                       466                                      0.485486  ...        -0.528808
4          False                       177                                      0.384474  ...        -0.530774
                                       169                                      0.496741  ...        -0.536888
                                       148                                      0.467030  ...        -0.543342
0          False                       1638                                     0.389118  ...        -0.543662
4          False                       112                                      0.263351  ...        -0.546907
                                       75                                       0.496741  ...        -0.547970
2   

In [20]:
viewer.fitness_evolution(points="all")

In [21]:
viewer.parameters_standardized_deviation()

In [22]:
viewer.shannon_entropy(bins=100_000)

In [24]:
fig = viewer.box_plot(3, nbest=500)
fig.show()

In [23]:
fig = viewer.parallel_coordinates(nbest=500, reversescale=True, unselected_opacity=0.05, uniformed=True)

for group in fig:
    display(group)

/Users/adm-lehodey/Documents/Workspace/Projects/Seapopym-optimisation/.venv/lib/python3.13/site-packages/sklearn/preprocessing/_data.py:2829: UserWarning: n_quantiles (1000) is greater than the total number of samples (500). n_quantiles is set to n_samples.
  warnings.warn(
